<a href="https://colab.research.google.com/github/hdhd7/MalashinLabs/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

class MonteCarloMassServiceSystem:
    def __init__(self):
        self.servers = [True, True]  #серверы: свободен или занят
        self.queue = []  #очередь клиентов
        self.customer_count = 0  #общее количество клиентов
        self.total_service_time = 0  #общее время обслуживания
        self.total_queue_wait_time = 0  #общее время ожидания в очереди
        self.total_system_wait_time = 0  #общее время ожидания в системе
        self.total_queue_length = 0  #общая длина очереди
        self.simulation_time = 1000  #время симуляции
        self.current_time = 0  #внутренние часы симуляции
        self.server_utilization = [0, 0]  #загрузка серверов

    def simulate(self, lambda_, mu):
        #основной метод симуляции системы массового обслуживания
        while self.current_time < self.simulation_time:
            self.current_time += 1  #увеличиваем текущее время на 1

            #поступление новых клиентов
            self.generate_customers(lambda_)

            #обработка клиентов серверами
            self.process_servers(mu)

            #проверка занятости серверов и увеличение времени в системе
            if self.queue and not any(self.servers):
                self.total_system_wait_time += self.current_time - self.queue[0]  #если оба сервера заняты

            #освобождение серверов
            self.release_servers()

        #вывод результатов
        self.print_results(lambda_, mu)

    def generate_customers(self, arrival_rate):
        #генерация новых клиентов и добавление их в очередь
        for _ in range(arrival_rate):
            self.queue.append(self.current_time)  #клиент поступает в очередь
            self.customer_count += 1  #увеличиваем счетчик клиентов

    def process_servers(self, service_rate):
        #обработка клиентов серверами, если они свободны
        for i in range(2):
            if self.servers[i] and self.queue:
                customer_arrival_time = self.queue.pop(0)  #извлечение клиента из очереди
                service_time = 1 / service_rate  #время обслуживания клиента
                self.total_service_time += service_time  #добавляем время обслуживания

                #обработка времени ожидания клиента в очереди и системе
                if self.queue:
                    self.total_queue_wait_time += self.current_time - self.queue[0]  #время ожидания в очереди
                    self.total_system_wait_time += self.current_time - self.queue[0]  #время в системе

                self.servers[i] = False  #занять сервер
                self.server_utilization[i] += service_time + (self.current_time - customer_arrival_time)  #учет загрузки сервера

                #увеличиваем общее время ожидания в системе, если клиент ожидает долго
                if random.random() < 0.6:
                    self.total_system_wait_time += self.current_time - customer_arrival_time

    def release_servers(self):
        #освобождение серверов с вероятностью
        for i in range(2):
            if not self.servers[i]:
                if random.random() < 0.2:  #вероятность освобождения сервера
                    self.servers[i] = True  #освобождаем сервер
                    self.total_queue_length += len(self.queue)  #учитываем длину очереди

    def calculate_probabilities(self, lambda_, mu):
        #вычисление вероятностей состояний системы с учетом λ и μ
        rho = lambda_ / (2 * mu)  #коэффициент загрузки одного сервера

        #вероятность, что в системе 0 клиентов (оба сервера свободны)
        P0 = 1 / (1 + (lambda_ / mu) + (lambda_ ** 2) / (2 * mu ** 2))

        #вероятность, что в системе 1 клиент (один сервер свободен)
        P1 = (lambda_ / mu) / (1 + (lambda_ / mu) + (lambda_ ** 2) / (2 * mu ** 2))

        #вероятность, что в системе 2 или более клиентов (оба сервера заняты)
        P_busy = 1 - (P0 + P1)

        return P0, P1, P_busy

    def print_results(self, lambda_, mu):
        #вывод результатов симуляции
        avg_queue_wait_time = self.total_queue_wait_time / self.customer_count if self.customer_count > 0 else 0  #среднее время ожидания в очереди
        avg_system_wait_time = self.total_system_wait_time / self.customer_count if self.customer_count > 0 else 0  #среднее время в системе
        avg_queue_length = self.total_queue_length / self.current_time if self.current_time > 0 else 0  #средняя длина очереди
        system_utilization = sum(self.server_utilization) / (self.simulation_time * 2)  #загрузка серверов

        #вывод средних значений
        print(f"Среднее время ожидания в очереди: {avg_queue_wait_time:.2f}")
        print(f"Среднее время ожидания в системе: {avg_system_wait_time:.2f}")
        print(f"Средняя длина очереди: {avg_queue_length:.2f}")
        print(f"Загрузка системы: {system_utilization:.2f}")

        #вычисление и вывод вероятностей
        P0, P1, P_busy = self.calculate_probabilities(lambda_, mu)
        print(f"Вероятность, что оба сервера заняты: {P_busy:.2f}")
        print(f"Вероятность, что один сервер свободен: {P1:.2f}")
        print(f"Вероятность, что оба сервера свободны: {P0:.2f}")

#запуск симуляции
lambda_ = 5  #интенсивность поступления клиентов (λ)
mu = 3  #интенсивность обслуживания (μ)
system = MonteCarloMassServiceSystem()
system.simulate(lambda_, mu)

Среднее время ожидания в очереди: 35.00
Среднее время ожидания в системе: 149.63
Средняя длина очереди: 874.57
Загрузка системы: 87.60
Вероятность, что оба сервера заняты: 0.34
Вероятность, что один сервер свободен: 0.41
Вероятность, что оба сервера свободны: 0.25
